# Star Destroyers - Exploring OEIS A117966 in Python / p5

>"If you look at the graph, it looks like this. It looks like *Star Wars*, a scence from *Star Wars*."

I first tried plotting something from OEIS after watching Neil Sloane on [Numberphile](https://www.youtube.com/watch?v=pAMgUB51XZA&list=PLt5AfwLFPxWLkoPqhxvuA8183hh1rBnGM&t=472s&ab_channel=Numberphile). One very neat looking parallelagram plot is shown in that video made with the primes manipulated in binary. 

Another plot [from the third video](https://www.youtube.com/watch?v=o8c4uYnnNnc&ab_channel=Numberphile) in the series is even more suprising - an evil looking array of Imperial Star Destroyers. Lots and lots of them, and in formation! 

![alt text](https://github.com/mrcordiner/processing-sketches/raw/main/oeis/oeis-a117966.png)

This image is teased out of base 3 numbers. Many students will be familar with binary numbers, but likely not have done much with base 3, or ternary, numbers. The math might seem a bit strange at first but if you remember place value then base 3 numbers are just like decimal or binary numbers but with columns of different 'weights'.


This notebook will focus first on working with ternary numbers and some additional ideas. Then it will examine how the series is created in detail.

Briefly the series is composed of the positive integers, which are written as a ternary number - which have the digits 0,1,2. These numbers are manipulated by changing any 2s into -1s. This fancy new result is then considered a number using "Balanced Ternary" where each column can only have the digtis 1,0,-1. This value is converted back to decimal and plotted.

Details and step-by-step breakdown and coding follows.

https://oeis.org/A117966

This notebook runs in the browser using Basthon - see [these instructions](https://github.com/mrcordiner/processing-sketches/tree/main/oeis)

## Python / p5 Version using Basthon Notebook
### Converting to Base 3

Ternary are numbers use base 3. So numbers, will only contain the digits 0,1,2. 

Counting in ternary looks like this:

| Decimal | Ternary |
| ----------- | ----------- |
| 0 | 0 |
| 1 | 1 |
| 2 | 2 |
| 3 | 10 |
| 4 | 11 |
| 5 | 12 |
| 6 | 20 |

The rightmost column in ternary is the 1s column and has the weight of 1, similar to decimal or binary. There can be a 0, 1 or 2 in this column. The next column has a weight of 3, so counting up from 0 looks like 0, 1, 2, 10. The 1 is in the 3s column, which is 3 raised to the power of 1. The next column is 3 raised to the powr of 2 or the 9s column. The 1s column is simply 3 raised to the power of 0, which is 1 for any number.

Decimal 5 is 12 in ternary because it is a 1 in the 3s column and a 2 in the 1s column or 1 x 3 + 2 x 1 = 5.

A simple way to convert from decimal to ternary is to use Numpy.

In [2]:
import numpy
numpy.base_repr(210, base=3)

'21210'

Let's build our own function to understand the process - start with going from ternary to decimal. Eg. 21210 in ternary would be 0 x 3^0 + 1 x 3^1 + 2 x 3^2 + 1 x 3^3 + 2 x 3^4 = 210

In [3]:
n = "21210"

i=4 # start at the rightmost column
for power in range(5):
    print(n[i], "x 3 ^", power, "=", int(n[i])*3**power)
    i-=1 # move left

0 x 3 ^ 0 = 0
1 x 3 ^ 1 = 3
2 x 3 ^ 2 = 18
1 x 3 ^ 3 = 27
2 x 3 ^ 4 = 162


In [4]:
n = "21210"

# this time collect the sum
s = 0

i=4 # start at the rightmost column
for power in range(5):
    s+=int(n[i])*3**power
    i-=1 # move left

print(s)

210


In [5]:
def tern_to_dec(n):
    # now a function!
    # accepts a decimal and returns a ternary number as a string
    s = 0
    n = str(n)

    i=4 # start at the rightmost column

    for power in range(5):
        s+=int(n[i])*3**power
        i-=1 # move left

    return(s)

In [6]:
tern_to_dec(21210)

210

## Converting from Decimal to Ternary

First we will find the largest power of 3 that a decimal number might contain. So what is the largest power of 3 that is greater then the given number? Then, since that power of 3 is too large we will start at the next smaller power.

In [7]:
n = 210

# greedy algo - ie. left to right 
# first find largest power of 3 
max_i = 0
while 3**max_i < n:
    max_i += 1
print(max_i)

5


Now that we know which exponent to start with we start a loop to start break the decimal number down into powers of three (possibly doubled).

In [8]:
def dec_to_tern(n):
    # take a decimal return ternary as string
    # greedy algo - ie. left to right 
    
    # first find largest power of 3 
    max_i = 0
    while 3**max_i < n:
        max_i += 1

    max_i -= 1
    
    #build new result
    result = ""
    
    for i in range(max_i,-1,-1):
        # test each power of 3 down to 0
        # 2
        if n >= 3**i * 2:
            result+="2"
            n -= 3**i * 2
        # 1
        elif n >= 3**i:
            result+="1"
            n -= 3**i
        # 0
        else:
            result+="0"

    return result        

In [9]:
dec_to_tern(210)

'21210'

## Balanced Ternary

So the interesting series we are about to plot is generated by taking ternary numbers and applying a simple rule to them - we are to take the ternary number and change all the 2s in it to -1s. 

Normally you wouldn't have -1s in the number, nor would you have a negative signs multiple times in a number. But this new number is going to be considered a balanced ternary number, which is different from ternary in that it can contain the digits 1,0 and -1. If a column has a -1 then it is a negative weight and is subtracted from the overall number. To keep things working in columns instead of -1 we will use a T. 

So a number 7 in decimal would be 1T1 -> +9−3+1

First let's code a function to accomplish the swap of 2s to Ts.


In [10]:
def swap(n):
    s = ""
    for ch in n:
        if ch == "2":
            s += "T"
        else:
            s += ch
    return s

In [11]:
print(swap('21210'))

T1T10


Next we need to calculate balanced ternary back to decimal.

In [12]:
def bal_tern_to_dec(n):
    p = len(n)-1
    num = 0
    for ch in n:
        if ch == "T":
            num -= 3**p
        else:
            num += 3**p * int(ch)
        p-=1
    return num

In [13]:
bal_tern_to_dec("1T1")

7

## P5 Plot

Now let's plot! We will plot all the values for x from 0 to 200000. We will scale the drawing to fit into the canvas.

Import p5 and also setup some global variables that can be tweeked to get things fitting in the canvas.

In [14]:
from p5 import *

Some values to scale the plot. You can alter this when the sketch is running to tweek the display.

In [15]:
sc = (0.002,0.0015)

Draw the plot using p5. Set up the required Processing functions.

In [16]:
def setup():
    createCanvas(900, 700)


def draw():
    background(11, 25, 48)
    translate(0,200)
    
    scale(sc)
    
    stroke(204, 240, 237)
    strokeWeight(80)
    
    # plot 
    for x in range(1000000):    
        y = bal_tern_to_dec(swap(dec_to_tern(x)))
        ellipse(x,y,20,20)
    
    noLoop() # no update

## Plot of OEIS A265326 up to 200000. 
 Will render [this image](https://github.com/mrcordiner/processing-sketches/blob/main/oeis/oeis-a265326.png) when run in the notebook.

In [17]:
run() # kick off the p5 code